In [1]:
# 전이학습의 일반적인 사용
# 대용량 데이터를 이용해 사전 학습한 모델을 소규모 데이터에 최적화 시킨다
# ImageNet  1천개의 사물에 대한 특징을 추출하도록 학습
# VGG16 - 마지막 분류기를 수정  3x3 합성곱, 최대폴링  CNN과 유사
# CIFAR-10

In [2]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg16

device = "cuda" if torch.cuda.is_available() else 'cpu'

model = vgg16(pretrained = True)
model

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
fc = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=10, bias=True)
  )

In [4]:
model.classifier = fc

In [5]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
# 라이브러리 로드
import tqdm
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip,RandomCrop,Normalize
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

In [16]:
training_data = CIFAR10(root = './', train=True, download=True, transform=ToTensor)
rgb_m = training_data.data.mean(axis=(0,1,2)) / 255
rgb_s = training_data.data.std(axis=(0,1,2)) / 255
rgb_m, rgb_s

Files already downloaded and verified


(array([0.49139968, 0.48215841, 0.44653091]),
 array([0.24703223, 0.24348513, 0.26158784]))

In [10]:
transforms = Compose([
    Resize(224),
    RandomCrop((232,232),padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),    
    Normalize(mean = rgb_m, std=rgb_s )
])

In [11]:
# 데이터 로더정의
training_data = CIFAR10(root = './', train=True, download=True, transform=transforms)
test_data = CIFAR10(root = './', train=False, download=True, transform=transforms)

train_loader = DataLoader(training_data, batch_size=32,shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# 학습 루프 정의
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)
for epoch in range(1):
  it = tqdm.tqdm(train_loader)
  for data, label in it:
    optim.zero_grad()
    preds = model(data.to(device))
    loss = nn.CrossEntropyLoss()(preds, label.to(device))
    loss.backward()
    optim.step()
    
    it.set_description(f"epoch:{epoch+1} loss:{loss.item()}")
torch.save(model.state_dict(),"/content/drive/MyDrive/Colab Notebooks/CIFAR_pretrained.pth")

epoch:1 loss:0.5691813230514526: 100%|██████████| 1563/1563 [16:23<00:00,  1.59it/s]


In [18]:
# 성능확인
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/CIFAR_pretrained.pth',map_location = device))

<All keys matched successfully>

In [19]:
num_corr = 0
with torch.no_grad():
  for data, label in test_loader:
    output = model(data.to(device))
    preds = output.data.max(1)[1]
    corr = preds.eq(label.to(device).data).sum().item()
    num_corr  += corr
print(f"accuracy : {num_corr / len(test_data)}")    

accuracy : 0.8941
